In [24]:
import numpy as np
import cv2
import os
import random
from skimage.feature import hog
from skimage import exposure
from sklearn import svm
# from sklearn.decomposition import PCA
# from skimage.transform import pyramid_gaussian

In [2]:
def leerFotos(path):
    os.chdir(path)
    listaPos = os.listdir()
    fotosDic = {}
    for i in range(len(listaPos)):
        fotosDic[i] = cv2.imread(listaPos[i],0)
    return fotosDic

In [3]:
path = 'C:\\Users\\David\\Desktop\\Human_Detection_Project-master\\train_pos'
fotosPosTrain = leerFotos(path)

path = 'C:\\Users\\David\\Desktop\\Human_Detection_Project-master\\train_neg'
fotosNegTrainSinTratar = leerFotos(path)

In [4]:
path = 'C:\\Users\\David\\Desktop\\Human_Detection_Project-master\\Test\\pos'
fotosTest = leerFotos(path)

In [5]:
testPos = fotosPosTrain[72]
testNeg = fotosNegTrainSinTratar[150]

# ------PRETRATAMIENTO DE LAS IMAGENES POSITIVAS-----

# Adaptamos el tamaño de las imagenes positivas a multiplo de 8

In [6]:
def recorta_imagen(imagen):    
    return imagen[:128,:64]

In [7]:
testPosRes = recorta_imagen(testPos)
for i in range(len(fotosPosTrain)):
    fotosPosTrain[i] = recorta_imagen(fotosPosTrain[i])

# Ampliamos el rango dinámico para obtener más contraste

In [8]:
def ampliacion(imagen):    
    r1 = np.min(imagen)
    r2 = np.max(imagen)      
    imagen = np.float32(imagen)
    imagen = 255*(imagen-r1)/(r2-r1)
    imagen = np.uint8(imagen)   
    return imagen

In [9]:
testPosRes = ampliacion(testPosRes)
for i in range(len(fotosPosTrain)):
    fotosPosTrain[i] = ampliacion(fotosPosTrain[i])

# Suavizamos las imagenes

In [10]:
kernelSize = 3
kernel = np.ones((kernelSize,kernelSize))/(kernelSize**2)
print(kernel)

[[0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]
 [0.11111111 0.11111111 0.11111111]]


In [11]:
testPosRes = cv2.filter2D(testPosRes,-1,kernel)
for i in range(len(fotosPosTrain)):
    fotosPosTrain[i] = cv2.filter2D(fotosPosTrain[i],-1,kernel)

# ------PRETRATAMIENTO DE LAS IMAGENES NEGATIVAS-----

# Tratamos las imagenes como anteriormente, ampliamos rango dinamico y suavizamos

In [12]:
for i in range(len(fotosNegTrainSinTratar)):
    fotosNegTrainSinTratar[i] = cv2.filter2D(ampliacion(fotosNegTrainSinTratar[i]),-1,kernel)

# De cada imagen, generamos unos trozos aleatorios del tamaño de las imagenes positivas

In [13]:
def generaImagenesRecortadas(dataPos,dataNeg):    
    numeroCortesxPh = 20      # Numero de recortadas aleatorias por foto
    rowsFotoPersona = dataPos[0].shape[0]
    colsFotoPersona = dataPos[0].shape[1]
    
    fotosNegTrain = {}
    for i in range(len(dataNeg)):      
        rowsFotoEntorno = dataNeg[i].shape[0]
        colsFotoEntorno = dataNeg[i].shape[1]
        dimCol = colsFotoEntorno - colsFotoPersona
        dimRow = rowsFotoEntorno - rowsFotoPersona
        
        for j in range(numeroCortesxPh):    
            randRow = random.randint(0,dimRow-1)
            randCol = random.randint(0,dimCol-1)
            randomImage = dataNeg[i][randRow:randRow + rowsFotoPersona ,randCol: randCol + colsFotoPersona]
            fotosNegTrain[j+numeroCortesxPh*i] = randomImage
            
    return fotosNegTrain

In [14]:
fotosNegTrain = generaImagenesRecortadas(fotosPosTrain,fotosNegTrainSinTratar)

# Creacion de los descriptores HOG

In [18]:
orientations = 9
pixels_per_cell = (8,8) 
cells_per_block = (2,2)
block_norm = 'L2'
visualize = False
visualise = None
transform_sqrt = True
feature_vector = True

In [19]:
def vectolizarHOG(imagenes,esPositivo):
    ### Esata funcion aplica HOG a todas las imagenes del diccionario
    
    #tamaño del vector
    tamVector = len(hog(imagenes[0],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector))

    #inicializamos la matriz que guarda los imagenes vectolizados
    fdImagenes = np.zeros((len(imagenes),tamVector))
    
    for i in range(len(imagenes)):
        fdImagenes[i,:] = hog(imagenes[i],orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
    
    #asignamos el vector de clases(y) 
    if esPositivo:
        y = np.ones((len(imagenes),1))
    else:
        y = np.zeros((len(imagenes),1))

    return fdImagenes,y

In [20]:
fdPosTrain,yPos = vectolizarHOG(fotosPosTrain,1)

In [21]:
fdNegTrain,yNeg = vectolizarHOG(fotosNegTrain,0)

In [22]:
fotosTrain = np.vstack((fdPosTrain,fdNegTrain))
y = np.vstack((yPos,yNeg))

# Entrenamos el modelo

In [25]:
clf = svm.LinearSVC(C = 0.007)
clf.fit(fotosTrain,y.ravel())

LinearSVC(C=0.007, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [26]:
clf.score(fdPosTrain,yPos)

0.9751332149200711

In [47]:
def imDetectado(im,clf,des,hPos,wPos):
    h,w = im.shape
    resultado = np.empty((h,w,3),dtype = np.uint8)
    resultado[:,:,0] = im
    resultado[:,:,1] = im
    resultado[:,:,2] = im    
    for i in range(0,h-hPos,des):
        for j in range(0,w-wPos,des):
            bloque = im[i:i+hPos,j:j+wPos]
            fBloque = hog(bloque,orientations,pixels_per_cell,cells_per_block,block_norm,visualize,visualise,transform_sqrt,feature_vector)
            if (clf.predict(fBloque.reshape(1,-1)) == 1):
                resultado[i,j:j+wPos,2] = 255
                resultado[i+hPos,j:j+wPos,2] = 255
                resultado[i:i+hPos,j,2] = 255
                resultado[i:i+hPos,j+wPos,2] = 255                
    return resultado

In [48]:
detect = imDetectado(fotosTest[0],clf,10,128,64)

In [ ]:
# cv2.imshow('Original',testPos)
# cv2.imshow('Tratada',testPosRes)
# cv2.imshow('per',hogIm)
print(fotosTest[0].shape)
cv2.imshow('Test',fotosTest[0])
cv2.imshow('Detectado',detect)
cv2.waitKey(0)
cv2.destroyAllWindows()

(605, 1060)
